In [1]:

from SSD_generate_anchors import generate_ssd_priors
from CustomDataLoader import DataAugmentor
from utility import *
from pathlib import Path
import collections

Using TensorFlow backend.


In [2]:
root = Path.home()/'data'/'VOCdevkit'/'VOC2007'

voc_2007_datafile = root/'ImageSets'/'Main'/'train.txt'
voc_2007_images   = root/'JPEGImages'
voc_2007_annotations = root/'Annotations'

In [3]:
SSDBoxSizes = collections.namedtuple('SSDBoxSizes', ['min', 'max'])

Spec = collections.namedtuple('Spec', ['feature_map_size', 'shrinkage', 'box_sizes', 
                                       'aspect_ratios'])

# the SSD orignal specs
specs = [
    Spec(38, 8, SSDBoxSizes(30, 60), [2]),
    Spec(19, 16, SSDBoxSizes(60, 111), [2, 3]),
    Spec(10, 32, SSDBoxSizes(111, 162), [2, 3]),
    Spec(5, 64, SSDBoxSizes(162, 213), [2, 3]),
    Spec(3, 100, SSDBoxSizes(213, 264), [2]),
    Spec(1, 300, SSDBoxSizes(264, 315), [2])
]

In [4]:
priors = generate_ssd_priors(specs)
testloader = DataAugmentor()

data       = testloader.flow_from_directory(root = root,
                                            data_file=voc_2007_datafile,
                                            target_size=300,
                                            batch_size = 4,
                                            shuffle    = True
                                            )

In [5]:
def intersect(box_a, box_b):

    """ We resize both tensors to [A,B,2] without new malloc:
    [A,2] -> [A,1,2] -> [A,B,2]
    [B,2] -> [1,B,2] -> [A,B,2]
    Then we compute the area of intersect between box_a and box_b.
    Args:
      box_a: (tensor) bounding boxes, Shape: [A,4].
      box_b: (tensor) bounding boxes, Shape: [B,4].
    Return:
      (tensor) intersection area, Shape: [A,B].
    """
    box_a = np.array(box_a, dtype = np.float32)
    box_b = np.array(box_b, dtype = np.float32) 
    
    A = box_a.shape[0]
    B = box_b.shape[0]
    
    box_a = box_a.reshape(A, 1, -1)
    box_a = box_a.repeat(B, axis = 1)
#     box_a = box_a.reshape(-1, 4)
    
    box_b = box_b.reshape(1,B, -1)
    box_b = box_b.repeat(A, axis = 0)
#     box_b = box_b.reshape(-1, 4)
    
    xy_min_a = box_a[:,:,:2]
    xy_min_b = box_b[:,:,:2]
    
    xy_max_a = box_a[:,:,2:]
    xy_max_b = box_b[:,:,2:]
    
    max_xy   = np.maximum(np.maximum(xy_min_a, xy_min_b), 0)    
    min_xy   = np.minimum(np.minimum(xy_max_a, xy_max_b), 1)

  
    tensor  = min_xy - max_xy
       
    intersection = tensor[:,:,0] * tensor[:,:,1]
    
    return intersection # Areas of intersection

def jaccard(box_a, box_b):
    
    intersection = intersect(box_a, box_b)
    print('intersection shape', intersection.shape)
    
    A = box_a.shape[0]
    B = box_b.shape[0]
    

    
    # area_box_a = (xmax - xmin) * (ymax - ymin)
    area_box_a  = (box_a[:,2] - box_a[:,0]) * (box_a[:,3] - box_a[:,1])
    
    area_box_a = area_box_a.reshape(-1, 1)
    area_box_a = area_box_a.repeat(B, axis = 1)
    
    # calculate areas of box B
    area_box_b  = (box_b[:,2] - box_b[:,0]) * (box_b[:,3] - box_b[:,1])
    
    area_box_b = area_box_b.reshape(1,-1)
    area_box_b = area_box_b.repeat(A, axis = 0)
    
    print(area_box_a.shape)
    print(area_box_b.shape)
    union       = area_box_a + area_box_b - intersection
    
    iou          = intersection/ union
    return iou

In [10]:
sample = data[0]

image, target = sample
batch_size = image.shape[0]

for i in range(batch_size):
    t = point_form(target[i][:,1:])
#     a = intersect(t, priors)
    b = jaccard(t, point_form(priors)) 
    if i == 0:
        break

intersection shape (2, 8732)
(2, 8732)
(2, 8732)


In [11]:
b.shape

(2, 8732)

In [12]:
np.max(b, axis=1)

array([358.80908844,  26.49491125])

In [13]:
np.min(b, axis=1)

array([-2217.43399654, -9698.6249966 ])